In [4]:
# importing libraries

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from nexinfosys import NISClient, display_visjs_jupyterlab
import io
import networkx as nx
import csv


In [5]:
def write_csv(output_csv,name): #eg flow_graph_solution.csv
    with open(name,'wb') as file:
        for line in output_csv:
            file.write(line)
    return name

In [6]:
# Input Nis Format File
fname = 'DatosGC_NIS3ToNIS - sin duplicados.xlsx'

### Connect to Nis Client

In [7]:
c = NISClient("https://one.nis.magic-nexus.eu/nis_api")
#To local:
#c = NISClient("http://localhost:5000/nis_api")
# Login, open session, load a workbook (which is in Nextcloud), submit (execute!)
c.login("test_user")
#print("Logged in")

True

In [8]:
c.open_session()
#print("Session opened")

True

In [9]:
n = c.load_workbook(fname)
print("N worksheets: "+str(n))

N worksheets: 10


In [10]:
r = c.submit()

In [11]:
any_error = False
if len(r) > 0:
    for i in r:
        if i["type"] == 3:
            any_error = True
            print(str(i))
            

In [12]:
if not any_error:
    # Obtain available datasets
    r = c.query_available_datasets()
    if len(r) > 0:
        results = {}
        for ds in r:
            results[ds["name"]] = {d["format"].lower(): d["url"] for d in ds["formats"]}


In [13]:
#See Warnings
#pd.DataFrame(r)

### Available Resuts

In [14]:
pd.DataFrame(c.query_available_datasets())

,description,formats,name,type
0,"Parameter values per Scenario [4 rows, 12 cell...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",params,dataset
1,"Flow Graph Solver - Interfaces [1624 rows, 292...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution,dataset
2,"Graph of Interfaces, Quantities; Scales and Ex...","[{'format': 'VisJS', 'url': '/nis_api/isession...",interfaces_graph,graph
3,Processors and exchanges graph,"[{'format': 'VisJS', 'url': '/nis_api/isession...",processors_graph,graph
4,Dictionary of Sankey Graph for every scenario ...,"[{'format': 'JSON', 'url': '/nis_api/isession/...",Sankey_Graph,Graph
5,Processors,"[{'format': 'GeoJSON', 'url': '/nis_api/isessi...",processors_geolayer,geolayer
6,Model,"[{'format': 'JSON', 'url': '/nis_api/isession/...",Model,model
7,OWL ontology,"[{'format': 'OWL', 'url': '/nis_api/isession/r...",Ontology,ontology
8,Python script,"[{'format': 'Python', 'url': '/nis_api/isessio...",Python script,script
9,R script,"[{'format': 'R', 'url': '/nis_api/isession/rse...",R script,script


In [15]:
#TODO widget para elegir output y descagarlo

## Exploring Flow Graph Solution

In [16]:
flow_graph_solution = c.download_results([(results["flow_graph_solution"]['csv'])])
write_csv(flow_graph_solution,"flow_graph_solution.csv")
df = pd.read_csv("flow_graph_solution.csv")

In [36]:
df.head()

,Scenario,Period,Scope,Conflict_Partof,Conflict_Itype,Processor,Interface,Orientation,RoegenType,Value,Computed,Observer,Expression,Unit,Level,System,Subsystem,Sphere
0,default,2019,Internal,No,No,G,Agua,Input,Flow,1945548.0,Yes,NaN,((G.Convencional:Agua:Input)+(G.Ecologico:Agua...,m3,n,GC,Local,Technosphere
1,default,2019,Internal,No,No,G,AsalariadosTC,Input,Fund,738720.0,Yes,NaN,((G.Convencional:AsalariadosTC:Input)+(G.Ecolo...,hr,n,GC,Local,Technosphere
2,default,2019,Internal,No,No,G,DES90yREU,Input,Flow,271112.5,Yes,NaN,G.Integrado:DES90yREU:Input,m3,n,GC,Local,Technosphere
3,default,2019,Internal,No,No,G,HA,Input,Fund,885840.0,Yes,NaN,((G.Convencional:HA:Input)+(G.Ecologico:HA:Inp...,hr,n,GC,Local,Technosphere
4,default,2019,Internal,No,No,G,LU,Input,Fund,235.5,Yes,NaN,((G.Convencional:LU:Input)+(G.Ecologico:LU:Inp...,ha,n,GC,Local,Technosphere


### Bar Plot By Level

In [35]:
fig = go.FigureWidget( layout=go.Layout() )
@interact(Level = list(set(df['Level'])), 
          Scenario = list(set(df['Scenario'])),
          Period = list(set(df['Period'])),
          Scope = list(set(df['Scope'])),
          Interface = list(set(df['Interface']))         
         )
def Bar_chart_Level(Level,Scenario,Period,Scope,Interface):
    data = pd.read_csv("flow_graph_solution.csv")
    data = data[(data['Scenario'] == Scenario)&(data['Level'] == Level)&(data['Period'] == Period)&(data['Scope'] == Scope)&(data['Interface'] == Interface)]
    y_pos =np.arange(len(data.Processor))
    plt.bar(y_pos,data.Value)
    plt.xticks(y_pos,data.Processor, rotation = 90)
    plt.show()

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)   
    
    

interactive(children=(Dropdown(description='Level', options=('n-4', 'n-3', 'n-2', 'n-1', 'n'), value='n-4'), D…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …